## Preprocess

In [ ]:
import cv2
import numpy as np
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
train_labels = pd.read_csv('data/labels.csv')

In [ ]:
train_len = len(train_labels)
breeds = set(train_labels['breed'])
class_len = len(breeds)
class_to_num = dict(zip(breeds, range(class_len)))
num_to_class = dict(zip(range(class_len), breeds))

In [ ]:
dim = 299
X_train = np.zeros((train_len, dim, dim, 3),dtype=np.uint8)
y_train = np.zeros((train_len, class_len),dtype=np.uint8)
for i in tqdm(range(train_len)):
    image_path = 'data/train/{}.jpg'.format(train_labels['id'][i])
    X_train[i] = cv2.resize(cv2.imread(image_path), (dim,dim))
    y_train[i][class_to_num[train_labels['breed'][i]]] = 1

In [ ]:
plt.figure(figsize=(12,6))
for i in range(8):
    index = randint(0,train_len-1)
    plt.subplot(2,4,i+1)
    plt.imshow(X_train[index][:,:,::-1])
    plt.title(num_to_class[y_train[index].argmax()])

## Keras Implementation

In [ ]:
from keras.layers import Input, Dense, Lambda, Flatten, GlobalAveragePooling2D, Dropout, Activation
from keras.models import Model, Sequential
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

In [ ]:
def feature_extraction(base_model, X):
    model = base_model(include_top=False, input_shape=(dim,dim,3), weights='imagenet')
    inputs = Input((dim, dim, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = model(x)
    x = GlobalAveragePooling2D()(x)
    model = Model(inputs,x)
    features = model.predict(X, batch_size=1, verbose=1)
    return features

In [ ]:
inception_features_train = feature_extraction(InceptionV3, X_train)
np.save('data/features/inception_features_train.npy', inception_features_train)

In [ ]:
xception_features_train = feature_extraction(Xception, X_train)
np.save('data/features/xception_features_train.npy', xception_features_train)

In [ ]:
#resnet_features_train = feature_extraction(ResNet50, X_train)
#np.save('data/features/resnet_features_train.npy', resnet_features_train)

In [ ]:
inception_features_train = np.load('data/features/inception_features_train.npy')
xception_features_train = np.load('data/features/xception_features_train.npy')
vgg19_features_train = np.load('data/features/vgg19_features_train.npy')
#resnet_features_train = np.load('data/features/resnet_features_train.npy')

In [ ]:
features = np.concatenate([inception_features_train, xception_features_train, vgg19_features_train], axis=-1)

In [ ]:
features = inception_features_train

In [ ]:
features.shape

In [ ]:
model = Sequential()
model.add(Dropout(0.5, input_shape=features.shape[1:]))
model.add(Dense(class_len, activation='softmax'))

adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
h = model.fit(features, y_train, batch_size=256, epochs=200, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')

In [ ]:
sample = pd.read_csv('data/sample.csv')
test_len = len(sample)
X_test = np.zeros((test_length, dim, dim, 3), dtype=np.uint8)
for i in tqdm(range(test_len)):
    image_path = 'data/test/{}.jpg'.format(sample['id'][i])
    X_test[i] = cv2.resize(cv2.imread(image_path), (dim,dim))

In [ ]:
inception_features_test = feature_extraction(InceptionV3, X_test)
np.save('data/features/inception_features_test.npy', inception_features_test)

In [ ]:
xception_features_test = feature_extraction(Xception, X_test)
np.save('data/features/xception_features_test.npy', xception_features_test)

In [ ]:
vgg19_features_test = feature_extraction(VGG19, X_test)
np.save('data/features/vgg19_features_test.npy', vgg19_features_test)

In [ ]:
resnet_features_test = feature_extraction(ResNet50, X_test)
np.save('data/features/resnet_features_test.npy', resnet_features_test)

In [ ]:
y_pred = model.predict(inception_features_test, batch_size=64)

In [ ]:
for i in breed:
    sample[i] = y_pred[:,class_to_num[i]]

In [ ]:
sample.to_csv('v2.csv', index=None)